In [1]:
import pandas as pd
import numpy as np
import random

from sklearn.model_selection import train_test_split

data = pd.read_csv("data/training_set_VU_DM_2014.csv")
data.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,NaN,NaN,NaN,NaN,-1.0,0.0,5.0,0,NaN,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0


In [ ]:
# Downsample a series so that there are as many non-clicked items as clicked and booked
def downsample_series(s):
    s1 = s.where(lambda e: e["click_bool"] == 1).dropna()
    s2 = s.where(lambda e: e["click_bool"] == 0).dropna()
    if (len(s2) == 0):
        return s1.sort_index()
    
    s2 = s2.sample(frac=min(1, len(s1)/len(s2)))
    return s1.append(s2).sort_index()

# Split on search ids
srch_ids = data["srch_id"].unique()
srch_ids_train, srch_ids_test_val = train_test_split(srch_ids, test_size=0.3, random_state = 42)
srch_ids_test, srch_ids_val = train_test_split(srch_ids_test_val, test_size=0.5, random_state = 42)

train = data.loc[data.srch_id.isin(srch_ids_train)]
val = data.loc[data.srch_id.isin(srch_ids_val)]
test = data.loc[data.srch_id.isin(srch_ids_test)]

# Downsample the training set
train = train.groupby("srch_id").apply(downsample_series).reset_index(drop=True)
train